In [85]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt, gc, os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet
/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [86]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import os
import lightgbm
import matplotlib.pyplot as plt
import seaborn as sbn
import numpy as np
import pandas as pd
import warnings
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, RocCurveDisplay, accuracy_score
import shap
import itertools

In [87]:
train = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet").groupby('customer_ID').tail(4)
test = pd.read_parquet("/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet").groupby('customer_ID').tail(4)
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
train.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-12-04,0.868580,0,0.001930,1.007504,0.001772,0.100470,0.0,0.006073,...,-1,-1,-1,0,0,0.0,NaN,0,0.004753,0
10,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-01-11,0.909811,0,0.002829,1.004798,0.008175,0.098882,0.0,0.003238,...,-1,-1,-1,0,0,0.0,NaN,0,0.003004,0
11,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-02-21,0.937349,0,0.019837,1.008307,0.000607,0.102985,0.0,0.007454,...,-1,-1,-1,0,0,0.0,NaN,0,0.006346,0
12,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,...,-1,-1,-1,0,0,0.0,NaN,0,0.002970,0
22,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2017-12-29,0.861109,10,0.006711,0.819772,0.007966,0.139138,0.0,0.005235,...,-1,-1,-1,0,0,0.0,NaN,0,0.003133,0


In [88]:
#shapes
train.shape, test.shape, train_labels.shape

((1802318, 190), (3656852, 190), (458913, 2))

### Handling missing values

In [89]:
#Checking Missing values having more than 45  ,%
removing_columns = train.columns[(train.isna().sum()/len(train))*100>45]

#Drop the missing columns having more than 40%
train = train.drop(removing_columns, axis=1)
test = test.drop(removing_columns, axis=1)

In [90]:
# Get only the numeric values
numeric_cols = train.select_dtypes(include=[np.number])

def fill_missing_value(dataset):
    for column in numeric_cols:
        dataset[column] = dataset[column].fillna(dataset[column].median())
    return dataset

miss_handled_train = fill_missing_value(train)
miss_handled_test = fill_missing_value(test)

In [91]:
#fill the missing values
miss_handled_train = train.bfill(axis='rows').ffill(axis='rows')
miss_handled_test = test.bfill(axis='rows').ffill(axis='rows')
train = miss_handled_train
test = miss_handled_test

In [92]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [93]:
train =train.groupby('customer_ID').tail(1)
test = test.groupby('customer_ID').tail(1)

In [94]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [95]:
#shape
train.shape, train_labels.shape, test.shape

((458913, 173), (458913, 2), (924621, 173))

**Data Preparation**

In [96]:
train = train.merge(train_labels, how='inner', on="customer_ID")

In [97]:
train.head()

,index,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,...,D_136,D_137,D_138,D_139,D_140,D_141,D_143,D_144,D_145,target
0,12,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,...,-1,-1,-1,0,0,0.0,0,0.002970,0,0
1,25,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,...,-1,-1,-1,0,0,0.0,0,0.003169,0,0
2,38,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880875,0,0.004284,0.812649,0.006450,0.165247,0.0,...,-1,-1,-1,0,0,0.0,0,0.000834,0,0
3,51,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,...,-1,-1,-1,0,0,0.0,0,0.005560,0,0
4,64,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.871900,0,0.007679,0.815746,0.001247,0.165247,0.0,...,-1,-1,-1,0,0,0.0,0,0.006944,0,0


In [98]:
test_data = test.copy()
train = train.drop(['index','customer_ID', 'S_2'], axis=1)
test = test.drop(['index','customer_ID', 'S_2'], axis=1)

### Encoding

In [99]:
#one hot Encoding for categorical features
obj_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
train = pd.get_dummies(train, columns=obj_col, drop_first=True)
test = pd.get_dummies(test, columns=obj_col, drop_first=True)
train.shape, test.shape

((458913, 199), (924621, 198))

### Normalizing data

In [100]:
train.head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_43,D_44,...,D_64_0,D_64_2,D_64_3,D_66_1,D_68_1,D_68_2,D_68_3,D_68_4,D_68_5,D_68_6
0,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.0,0.007174,0.091563,0,...,1,0,0,0,0,0,0,0,0,1
1,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.0,0.005068,0.060646,0,...,1,0,0,0,0,0,0,0,0,1
2,0.880875,0,0.004284,0.812649,0.006450,0.165247,0.0,0.007196,0.091563,0,...,0,1,0,0,0,0,0,0,0,1
3,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.0,0.009937,0.046104,0,...,1,0,0,0,0,0,1,0,0,0
4,0.871900,0,0.007679,0.815746,0.001247,0.165247,0.0,0.005528,0.044671,0,...,1,0,0,1,0,0,0,0,0,1


In [101]:
Features=train.loc[:, test.columns]
target = train['target']

In [102]:
#Normalizing data Using The z-score method
def normalize_data(dataset):
    for column in dataset.columns:
        dataset[column] = (dataset[column] - dataset[column].mean()) / dataset[column].std() 
    return dataset

Features = normalize_data(Features)
test = normalize_data(test)

**Model Building and Evaluation**

In [131]:
from lightgbm import LGBMClassifier

def LGBMModel(Feature, target):
    lgbm = LGBMClassifier(random_state=5)
    lgbm.fit(Features, target)
    return lgbm

In [104]:
from sklearn.neighbors import KNeighborsClassifier

def KNeighborsModel(Features, target):
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(Features, target)
    return neigh

In [134]:
def XGBModel(Features, target):
    xgb = XGBClassifier(n_estimators=300, max_depth=7, learning_rate=0.1).fit(Features, target)
    return xgb

In [119]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

def SVMLinearModel(Features, target):
    svmLinear = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))
    svmLinear.fit(Features, target)
    return svmLinear
    

In [ ]:
# Select the final model
# Knn_model = KNeighborsModel(Features, target)
# lgbm_model = LGBMModel(Features, target)
XGB = XGBModel(Features, target)
# svmModel = SVMLinearModel(Features, target)

final_model = XGB

In [ ]:
import joblib

# save
joblib.dump(final_model, "model.joblib")

In [ ]:
# Cross validation
np.mean(cross_val_score(XGB, Features, target, scoring='accuracy', cv=5))

In [ ]:
y_pred = final_model.predict(Features)
accuracy_score(target, y_pred)

In [ ]:
cm=confusion_matrix(target, y_pred)
plt.figure(figsize=(13,5))
plt.title("Confusion Matrix")
plt.imshow(cm, alpha=0.5, cmap='PuBu')
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment="center")
plt.show()

In [ ]:
print(classification_report(target, y_pred))

In [ ]:
test_data['prediction']=final_model.predict_proba(test)[:,1]
# test_data['prediction']=final_model._predict_proba_lr(test)[:,1]
test_data[['customer_ID','prediction']].to_csv("xgb.csv", index=False)